# **인공지능 기초 - 중간고사 대체 과제**

**<div style="text-align: right"> 202021497 이주철 </div>**

## **OpenAI API 기반 문제 해결 프로젝트**

### **과제 목적:**

OpenAI API를 사용하여 실생활에서 유용한 AI 시스템을 설계하고 Python 으로 구현하며, 입력(prompt) 설계 및 테스트를 통해 결과를 해석하고 보고서로 정리합니다.

### **과제:**

- OpenAI API 키를 활용해서 문장을 입력받았을 때 문체를 바꾸어 주는 시스템을 구현합니다. 
    - 예시 입력: "나는 오늘 학교에서 재밌게 놀았다." → 뉴스 기사체로 변환
    - 예시 출력: "오늘 오후, 한 초등학생이 학교에서 즐거운 시간을 보냈다."

위 예시를 기반으로 OpenAI 모델을 통해 입력/출력의 구성 방식을 구현한 뒤 테스트 및 실험으로 결과 비교 및 분석을 진행합니다. 
    
<br/>

### **문제 정의 및 목표**
 1) 사용자가 작성한 문장을 원하는 문체로 자동 변환하는 것은 쉽지 않습니다. 단어 선택, 어조, 문장 구조를 모두 스스로 바꿔야 하는 수고가 많아지는 경험을 합니다. 일기체, 블로그체 같은 문체는 자유 형식으로 작성이 되지만, 뉴스 기사체나 보고서 같은 문체는 그에 맞는 형식적인 규칙을 적용하여 변환해야 합니다. 한 가지 문제점으로는 원문에 없는 정보를 덧붙이거나 과장하는 문제가 있긴 합니다. 
 - 위 문제는 사용자가 입력한 문장의 내용은 유지하면서 원하는 문체로만 표현하게 합니다. 추가 정보 없이 정확한 스타일 변화만 수행하도록 프롬프트를 최적화할 예정입니다. 
 2) 학생이 레포트를 작성하는 상황에서 일반 문장을 기사체로 바꿔줄 수 있습니다. 블로그체나 일기체는 물론 작가나 블로거가 감성적인 글이나 차분한 느낌을 주고 싶을 때는 유용하게 쓰일 것입니다. 어린이 대상으로 콘텐츠를 작성할 때 복잡한 언어와 구조를 간단히 변환하여 읽기 편한 구조로 바꿔줍니다.
 
 #### **목표: 사용자가 작성한 문장의 의미를 그대로 유지하면서 다양한 문체 스타일로 정확하게 변환하는 도구를 만드는 것입니다.**

---

과제를 시작하기 전, 과제에 필요한 모듈을 다운로드합니다.

In [56]:
!pip install openai
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **시스템 설계 개요**

## **1) OpenAI API 키 가져오기**

---

- OpenAI 객체를 생성해서 client로 저장한다. 모든 GPT 호출은 client를 통해 이루어집니다.

In [57]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# ✅ .env에서 API 키 불러오기
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

## **2) 글쓰기 스타일에 맞는 prompt 작성하기**

---

- 문체 스타일을 prompt에 작성하여서 AI가 사용자가 입력하는 문장을 잘 변환할 수 있도록 학습 시킵니다.

In [ ]:
# - **단, 입력 문장에 없는 정보를 새로 추가하지 마세요.** 를 추가함으로써 사용자가 입력한 정보외에 다른 정보를 추가해서 변환하지 않는다.

style_prompts = {
    "뉴스 기사체": """
당신은 뉴스 기사 작성자입니다. 아래 문장을 뉴스 기사 스타일로 바꿔주세요.

지켜야 할 규칙:
- 문장의 구조와 말투는 뉴스 기사처럼 객관적이고 간결하게 작성해 주세요.
- 주어는 '한 시민', '한 초등학생', '지역 주민' 등으로 바꿔주세요.
- 시간 표현은 '오늘 오후', '12일 오전' 등으로 자연스럽게 표현해 주세요.
- 감정 표현이나 추측성 단어는 제거해 주세요.
- 반드시 입력 문장에서 제공된 정보만 사용하고, 새로운 사실이나 상상을 추가하지 마세요.

예시 1:
입력: 나는 오늘 학교에서 친구들과 재밌게 놀았다. 공부할 때는 재미없는데 쉬는 시간만 되면 재밌어
오늘 오후, 한 초등학생이 학교에서 친구들과 함께 즐거운 시간을 보냈다.

예시 2:
입력: 나는 오늘 경주로 1박2일 여행을 친구와 같이 갔다. 경주에서 맛난 음식과 역사를 체험하고 왔다.
출력: 오늘, 한 시민이 친구와 함께 경주를 방문해 음식과 역사 체험을 한 것으로 전해졌다.

예시 3:
입력: 나는 오늘 대전에서 유명한 빵집을 다녀왔다. 유명해서 그런지 웨이팅을 하고 1시간 만에 들어갔다.
출력: 오늘 오후, 한 시민이 대전의 유명한 제과점을 방문해 긴 대기 끝에 입장했다.
""",

    "일기체": """
당신은 청소년 혹은 성인이 쓰는 일기처럼 문장을 바꿔주세요.

- 문장은 1인칭으로 유지하고, 감정을 자연스럽게 표현해 주세요.
- 일기처럼 친근하고 편안한 말투를 사용하세요.
- 반드시 입력 문장에서 제공된 정보만 사용하고, 새로운 사실이나 상상을 추가하지 마세요.

예시 1:
입력: 나는 오늘 경주로 1박2일 여행을 친구와 같이 갔다. 경주에서 맛난 음식과 역사를 체험하고 왔다.
출력: 오늘은 친구랑 1박 2일 동안 경주 여행을 다녀왔다. 맛있는 음식도 먹고 역사도 체험하면서 정말 알찬 하루였다.

예시 2:
입력: 나는 오늘 학교에서 친구들과 재밌게 놀았다. 공부할 때는 재미없는데 쉬는 시간만 되면 재밌어
오늘은 학교에서 친구들이랑 쉬는 시간마다 정말 재밌게 놀았다. 공부할 때는 재미없었지만, 쉬는 시간 덕분에 하루가 즐거웠다.

예시 3:
입력: 나는 오늘 대전에서 유명한 빵집을 다녀왔다. 유명해서 그런지 웨이팅을 하고 1시간 만에 들어갔다.
출력: 오늘은 대전의 유명한 빵집에 다녀왔다. 사람들이 많아서 한 시간이나 기다렸지만, 그래도 뿌듯했다.
""",

    "블로그체": """
당신은 블로그 글 작성자입니다. 아래 문장을 블로그 스타일로 바꿔주세요.

- 말투는 친근하고 설명하듯이 써 주세요.
- 독자에게 말하듯 자연스럽게 풀어서 표현하세요.
- 반드시 입력 문장에서 제공된 정보만 사용하고, 새로운 사실이나 상상을 추가하지 마세요.

예시 1:
입력: 나는 오늘 대전에서 유명한 빵집을 다녀왔다. 유명해서 그런지 웨이팅을 하고 1시간 만에 들어갔다.
출력: 오늘은 대전에서 정말 유명한 빵집에 다녀왔어요! 인기가 많아서 한 시간이나 기다려야 했지만, 드디어 들어갔답니다.

예시 2:
입력: 나는 오늘 경주로 1박2일 여행을 친구와 같이 갔다. 경주에서 맛난 음식과 역사를 체험하고 왔다.
출력: 오늘은 친구랑 1박 2일 경주 여행을 다녀왔어요. 맛있는 음식도 먹고 역사 체험도 하면서 정말 알찬 시간이었어요.

예시 3:
입력: 요즘 공부하느라 잠도 재대로 못 자고 있다. 장학금 받기 위해 성적을 올리는 게 목표니까
출력: 요즘은 장학금 목표로 열심히 공부 중이에요. 잠도 부족하지만, 목표가 있으니 버틸 수 있는 것 같아요!
""",

    "어린이 말투": """
아래 문장을 귀엽고 어린이 말투로 바꿔주세요.

- 문장은 짧고 쉽고 귀엽게 바꿔주세요!
- 말끝에 ‘~했어’, ‘~했지롱’, ‘~잉’ 등 말투를 넣어주세요.
- 반드시 입력 문장에서 제공된 정보만 사용하고, 새로운 사실이나 상상을 추가하지 마세요.

예시 1:
입력: 나는 오늘 학교에서 재밌게 놀았다.
출력: 오늘 학교에서 친구랑 놀았어! 진짜 진짜 재밌었지롱~!

예시 2:
입력: 나는 오늘 대전에서 유명한 빵집을 다녀왔다. 유명해서 그런지 웨이팅을 하고 1시간 만에 들어갔다.
출력: 오늘 대전에서 빵집 갔다왔어! 사람 많아서 기다렸다가 들어갔지롱~

예시 3:
입력: 요즘 공부하느라 잠도 재대로 못 자고 있다. 장학금 받기 위해 성적을 올리는 게 목표니까
출력: 요즘 공부 열심히 하고 있어~ 잠도 못 자서 피곤해잉... 장학금 꼭 받을 거야!
"""
}

## **3) 문체 스타일 변환 시 구성 요소**

---
- 사용한 모델은 gpt-4o-mini를 사용하였습니다. 
- 시스템이 이해할 수 있도록 명시적으로 지시어를 작성하였습니다. 예시를 작성하여 Few-shot 기법을 이용하였습니다.
- 출력 형식은 GPT의 응답 문장 중 content 항목에서 바로 추출하며 단일 문장으로 출력한다. 사용자 입력은 sentence 인자로 전달됩니다. style 인자에 따라 prompt를 선택합니다. 

- temperature: 수가 높을수록 창의력과 유연성을 더하고 낮을수록 문체만 바꾸고 문장 정보 고정의 안정성이 높아진다. 즉, 사용자가 입력한 문장을 논리적이고 일관되게 출력한다. 

In [59]:
# 사용자가 작성한 문장을 원하는 새 문체 스타일로 변환합니다. 
def convert_to_new_style(sentence: str, style: str) -> str:

    prompt = f"""
요구 사항: {style_prompts[style]}

입력문장: "{sentence}"
"""
    # GPT 응답 중 첫 번째를 선택합니다.
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[
            {"role": "system", "content": "너는 문체만 바꾸는 도우미야. 입력된 내용에서 벗어난 정보를 추가하지 마. 표현 방식만 바꿔줘."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3, # 입력 받은 문장을 안정적인 수준으로 일관된 문체 생성에 적합하게 출력합니다. 숫자가 높으면 비적합한 문장으로 변환됩니다.
        max_tokens = 100
    )

    # 아래 문장으로 텍스트를 정제합니다.
    return response.choices[0].message.content.strip()

## **4) 사용자 인터페이스 역할**

---

- 문체 변환 프로그램의 사용자 인터페이스 역할을 하는 코드입니다. 사용자가 문장을 입력하고 원하는 스타일을 선택하면 GPT 기반 함수 convert to new style을 호출해서 결과를 보여줍니다. 

In [4]:
# 해당 파일이 직접 실행될 때만 아래 코드가 실행됩니다. 
if __name__ == "__main__":
    while True:
        user_input = input("📝 변환할 문장을 입력하세요 (exit 입력 시 종료): ")
        if user_input.lower() == "exit": # 사용자가 exit를 입력하면 종료할 수 있습니다.
            break
        print("🎨 변환할 스타일을 선택하세요: 뉴스 기사체 / 일기체 / 블로그체 / 어린이 말투")
        selected_style = input("👉 스타일 입력: ").strip()
        if selected_style.lower() == "exit":
            break

        # 입력된 스타일이 style_prompt에 없는 값이면 경고를 출력합니다. 다시 처음으로 돌아가서 문장입력 및 스타일 입력을 반복합니다.
        if selected_style not in style_prompts: 
            print("⚠️ 올바른 스타일을 입력해주세요.\n")
            if selected_style.lower() == "exit":
                continue
        result = convert_to_new_style(user_input, selected_style)
        print(f"\n📥 입력 문장:\n{user_input}")
        print(f"\n🧾 [{selected_style}] 문장:")
        print(result)
        print("-" * 50)

🎨 변환할 스타일을 선택하세요: 뉴스 기사체 / 일기체 / 블로그체 / 어린이 말투


## **5) 테스트 및 실험**

---

- 문체 변환 프로그램의 사용자 인터페이스 역할을 하는 코드입니다. 사용자가 문장을 입력하고 원하는 스타일을 선택하면 GPT 기반 함수 convert to new style을 호출해서 결과를 보여줍니다. 

테스트로 4가지 입력을 하고 style에 맞는 문체를 변환해 보았습니다. 
입력한 문장은 평서문으로 작성하고 각기 다른 스타일로 변환을 해보았을 때 문체 변환이 잘 이루어진 것을 확인할 수 있습니다. 

<img src="results_pic/resultpic.png" alt="실행한 결과" width="600"/>

### 5-1) 결과 비교 분석

---

- 처음에 프롬프트 작성했을 때 아래 작성한 예시처럼 지시문만 작성하였습니다. 각 문체 스타일이 어떤 특징을 가졌는지 어떤 식으로 문장을 구상하면 되는지만 적었습니다. 
즉, Zero-shot 방식으로 작성하였습니다. 예시 없이 설명만 준 방식입니다.
-> 그 결과 아래 이미지처럼 사용자가 입력한 문장과 많이 다른 형태를 보였습니다. 
과장이 많고 입력한 문장에 없는 정보들 또한 추가가 되어 출력된 사례를 확인할 수 있습니다. 

---

<img src="results_pic/exampresult.png" alt="처음 실행한 결과" width="1200"/>

<img src="results_pic/examp.png" alt="처음 작성한 프롬프트 예시" width="900"/>


이를 개선하고자 프롬프트를 다시 작성하였고, 예시를 들어서 학습을 시켰습니다. 
이는 One-shot 방식으로 예시를 1개만 보여주어 학습하였습니다.

- 

<img src="results_pic/exampresult2.png" alt="두번쨰 실행한 결과" width="500"/>

<img src="results_pic/examp2.png" alt="두번쨰 작성한 프롬프트 예시" width="700"/>


### 5-2) 최종 결과
---

- 최종적으로는 Few-shot 방식이 알맞을 거로 생각하여 최종 프롬프트를 작성하였습니다. 
결과는 아래와 같이 문장 4개를 테스트해 보았을 때 각 문체 스타일에 맞게 출력되었습니다. 

<img src="results_pic/resultpic.png" alt="최종적으로 실행한 결과" width="600"/>

## **6) 결과 요약 및 고찰**

---

- 문체 변환 프로그램의 사용자 인터페이스 역할을 하는 코드입니다. 사용자가 문장을 입력하고 원하는 스타일을 선택하면 GPT 기반 함수 convert to new style을 호출해서 결과를 보여줍니다. prompt 작성한 예시들을 잘 학습하여 문체를 변환할 때 잘 작동해 주었습니다. 

- 뉴스 기사체를 출력할 때 뉴스는 정확하고 상세한 정보를 줘야 하다 보니 오늘이라고 입력이 되면 자동으로 시간까지 출력이 됩니다. 뉴스 기사체에서는 맞는 형식이지만 사용자의 입력 취지와는 약간 다르지 않냐 라는 생각이 들었습니다. 사실상 사용자 입장에서는 오전인지 오후인지 저녁인지는 모르는 것인데 출력으로는 시간이 보통 오후로 나와서 이 부분이 아쉬웠습니다. 

- OpenAI API를 처음 활용해 보았는데 본인이 직접 프롬프트 작성하고 예시를 들어서 더 자세하게 모델에게 적용해 원하는 답을 끌어내는데 신기했습니다. 더 다양하게 분석하고 자세하게 프롬프트를 작성하면 사용자가 편하게 활용할 수 있다고 생각합니다. 